In [ ]:
import json
import requests
import json
import matplotlib
import re
import pydot
from graphviz import Digraph
from graphviz import Source
from inference_utils import Utils
import networkx as nx
from networkx.readwrite import json_graph
from __future__ import print_function

In [ ]:
from enum import Enum

class EdgeStatus(Enum):
    wiktdata_error = 1
    double_edge_error = 2
    single_zero_success = 3
    no_edge_success = 4
    neutral = 5
    edge_not_found = 6
    node_not_found = 7
    zero_noedge_both_success = 8 

def append_wikidata(graph, triplet, rule, lemma):
    curr_edges = [
        (n1, n2, d['color']) for n1, n2, d in graph.edges(data=True)]

    if triplet['arg1'] is "novalue" or triplet['arg2'] is "novalue" or triplet['arg3'] is "novalue":
        return EdgeStatus.wiktdata_error
    
    a = nx.get_node_attributes(graph,'str_name')
    
    for n in a:
        if a[n] == lemma:
            nulledgecount = 0
            edgetoit = False
            for e in curr_edges:
                #megy e bele el
                if e[1] == n:
                    edgetoit = True
                    
                if e[0] == n:
                    #ketts el vizsgalata
                    for c in curr_edges:
                        if c[0] == e[1] and c[1] == n:
                            return EdgeStatus.double_edge_error
                        
                if rule == 0:
                    if e[2] == 0 and e[0] == n:
                        nulledgecount+=1
                        if triplet['arg3'] not in a:
                            graph.add_node(triplet['arg3']+'_', str_name=triplet['arg3'], expanded=True)
                            graph.add_edge(triplet['arg3']+'_', e[1], color=0)
                            
                if rule == 2:
                    if e[2] == 2 and e[0] == n:
                        nulledgecount+=1
                        if triplet['arg1'] not in a:
                            graph.add_node(triplet['arg1']+'_', str_name=triplet['arg1'], expanded=True)
                            graph.add_edge(triplet['arg1']+'_', e[1], color=0)

            if nulledgecount == 0:
                return EdgeStatus.edge_not_found
            if nulledgecount == 1 and edgetoit == True:
                return EdgeStatus.single_zero_success
            if edgetoit == False and nulledgecount != 1:
                return EdgeStatus.no_edge_success
            if nulledgecount == 1 and edgetoit == False:
                return EdgeStatus.zero_noedge_both_success
            
            return EdgeStatus.neutral

    return EdgeStatus.node_not_found

In [ ]:
'''TESZT!!!'''
'''
triplets = {}
graphs = {}
statuses = {}
count = 0
with open("togetherfiltered.json", "r+") as f:
    for l in f:
        line = eval(l)
        predicate = line['arg2']
        data = {'word':   predicate}
        data_json = json.dumps(data)
        payload = {'json_payload': data_json}
        headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

        r = requests.post("http://127.0.0.1:5000/wikidata", data=data_json, headers=headers)
        
        word_graph = r.json()['word']
        g_word_graph = json_graph.adjacency.adjacency_graph(word_graph)
        
        status = append_wikidata(g_word_graph, line, 2)
        
        triplets[count] = (line['arg1'], line['arg2'], line['arg3'])
        graphs[count] = g_word_graph
        statuses[count] = status
        count+=1
'''

In [ ]:
import statistics

In [ ]:
dict_predicates_raw = {}
with open('/home/adaamko/wikidata/wikidata_all_ids', 'r') as f:
    count = 0
    for line in f:
        d = eval(line)
        if d['arg2'] not in dict_predicates_raw:
            dict_predicates_raw[d['arg2']] = 0
        else:
            dict_predicates_raw[d['arg2']] += 1

In [956]:
len(dict_predicates_raw)

893

In [957]:
print("Teljes Dataset")
print(statistics.mean(dict_predicates_raw.values()))
print(statistics.pstdev(dict_predicates_raw.values()))
print(sum(dict_predicates_raw.values()))

Teljes Dataset
96660.2206047
818079.669687
86317577


In [ ]:
dict_predicates = {}
with open('/home/adaamko/wikidata/wikidata_all_with_labels_filtered', 'r') as f:
    count = 0
    for line in f:
        d = eval(line)
        if d['arg2'] not in dict_predicates:
            dict_predicates[d['arg2']] = 0
        else:
            dict_predicates[d['arg2']] += 1

In [958]:
len(dict_predicates)

195

In [959]:
import statistics

In [960]:
print("Amire alkalmazhato a 4lang")
print(statistics.mean(dict_predicates.values()))
print(statistics.pstdev(dict_predicates.values()))
print(sum(dict_predicates.values()))

Amire alkalmazhato a 4lang
100772.292308
566557.073168
19650597


In [ ]:
dict_predicates_filter_novalue = {}
with open('/home/adaamko/wikidata/wikidata_all_with_labels_filtered', 'r') as f:
    count = 0
    for line in f:
        d = eval(line)
        if d['arg1'] is "novalue" or d['arg2'] is "novalue" or d['arg3'] is "novalue":
            continue
        if d['arg2'] not in dict_predicates_filter_novalue:
            dict_predicates_filter_novalue[d['arg2']] = 0
        else:
            dict_predicates_filter_novalue[d['arg2']] += 1

In [961]:
print("Novalue-ek kiszurese utan")
print(statistics.mean(dict_predicates_filter_novalue.values()))
print(statistics.pstdev(dict_predicates_filter_novalue.values()))
print(sum(dict_predicates_filter_novalue.values()))

Novalue-ek kiszurese utan
72991.7641026
374664.761134
14233394


In [ ]:
dict_sentences = {}
with open('/home/adaamko/wikidata/wikidata_all_with_labels_filtered', 'r') as f:
    count = 0
    for line in f:
        d = eval(line)
        if d['arg1'] is "novalue" or d['arg2'] is "novalue" or d['arg3'] is "novalue":
            continue
        if d['arg2'] not in dict_sentences:
            dict_sentences[d['arg2']] = d

In [965]:
triplets = {}
statuses_zero = {}
graphs = {}
graphs_zero = {}
graphs_two = {} 
statuses_two = {}
count = 0

for line in dict_sentences.values():
    predicate = line['arg2']
    data = {'word':   predicate}
    data_json = json.dumps(data)
    payload = {'json_payload': data_json}
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}

    r = requests.post("http://127.0.0.1:5000/wikidata", data=data_json, headers=headers)

    word_graph = r.json()['word']
    lemma = r.json()['lem']
    g_word_graph_zero = json_graph.adjacency.adjacency_graph(word_graph)
    g_word_graph_two = json_graph.adjacency.adjacency_graph(word_graph)

    status_zero = append_wikidata(g_word_graph_zero, line, 0, lemma)
    status_two = append_wikidata(g_word_graph_two, line, 2, lemma)
    
    triplets[count] = (line['arg1'], line['arg2'], line['arg3'])
    graphs_zero[count] = g_word_graph_zero
    graphs_two[count] = g_word_graph_two
    statuses_zero[count] = status_zero
    statuses_two[count] = status_two
    count+=1

In [966]:
errors_zero = [k for k in statuses_zero.keys() if statuses_zero[k] in [EdgeStatus(2), EdgeStatus(6), EdgeStatus(7)]]
after_errors_zero = [k for k in statuses_zero.keys() if statuses_zero[k] not in [EdgeStatus(2), EdgeStatus(6), EdgeStatus(7)]]
success_zero = [k for k in statuses_zero.keys() if statuses_zero[k] in [EdgeStatus(3), EdgeStatus(4), EdgeStatus(8)]]
neutral_zero = [k for k in statuses_zero.keys() if statuses_zero[k] in [EdgeStatus(5)]]
print("Osszes predicate",len(statuses_zero))
print("0 tipusu hibak",len(errors_zero))
print("Maradt", len(after_errors_zero))
print("Ebbol nem tudunk mit mondani", len(neutral_zero))
print("Amire gondoljuk hogy jo", len(success_zero))

Osszes predicate 195
0 tipusu hibak 87
Maradt 108
Ebbol nem tudunk mit mondani 24
Amire gondoljuk hogy jo 84


In [967]:
errors_two = [k for k in statuses_two.keys() if statuses_two[k] in [EdgeStatus(2), EdgeStatus(6), EdgeStatus(7)]]
after_errors_two = [k for k in statuses_two.keys() if statuses_two[k] not in [EdgeStatus(2), EdgeStatus(6), EdgeStatus(7)]]
success_two = [k for k in statuses_two.keys() if statuses_two[k] in [EdgeStatus(3), EdgeStatus(4), EdgeStatus(8)]]
neutral_two = [k for k in statuses_two.keys() if statuses_two[k] in [EdgeStatus(5)]]
print("Osszes predicate",len(statuses_two))
print("2 tipusu hibak",len(errors_two))
print("Maradt",len(after_errors_two))
print("Ebbol nem tudunk mit mondani",len(neutral_two))
print("Amire gondoljuk hogy jo",len(success_two))

Osszes predicate 195
2 tipusu hibak 168
Maradt 27
Ebbol nem tudunk mit mondani 2
Amire gondoljuk hogy jo 25


In [968]:
errors_zero_predicates = [triplets[k][1] for k in errors_zero]
after_errors_zero_predicates = [triplets[k][1] for k in after_errors_zero]
success_zero_predicates = [triplets[k][1] for k in success_zero]
neutral_zero_predicates = [triplets[k][1] for k in neutral_zero]

errors_two_predicates = [triplets[k][1] for k in errors_two]
after_errors_two_predicates = [triplets[k][1] for k in after_errors_two]
success_two_predicates = [triplets[k][1] for k in success_two]
neutral_two_predicates = [triplets[k][1] for k in neutral_two]

In [969]:
def print_statistics(rule, pred):
    data = [dict_predicates_filter_novalue[k] for k in pred]
    mean = statistics.mean(data)
    variance = statistics.pstdev(data)
    sum_data = sum(data)
    print(rule,"\n predicate: ",len(pred),"\n tripletek: ",sum_data,"\n átlag: ",mean,"\n szórás: ",variance,)

In [970]:
'''Először 0 tipusu élekre előszures utan'''
print_statistics("0 tipusu elekre eloszor eloszures utan:" ,after_errors_zero_predicates)

0 tipusu elekre eloszor eloszures utan: 
 predicate:  108 
 tripletek:  9276533 
 átlag:  85893.8240741 
 szórás:  388140.295277


In [971]:
'''Majd 2 tipusu élekre előszures utan'''
print_statistics("2 tipusu elekre eloszures utan:", after_errors_two_predicates)

2 tipusu elekre eloszures utan: 
 predicate:  27 
 tripletek:  1425311 
 átlag:  52789.2962963 
 szórás:  99993.9924536


In [972]:
'''0 tipusu elekre szabalyok alkalmazasa utan'''
print_statistics("0 tipusu elekre szabalyok utan:", success_zero_predicates)

0 tipusu elekre szabalyok utan: 
 predicate:  84 
 tripletek:  8420734 
 átlag:  100246.833333 
 szórás:  437205.618177


In [973]:
'''2 tipusu elekre szabalyok alkalmazasa utan'''
print_statistics("2 tipusu elekre szabalyok utan:", success_two_predicates)

2 tipusu elekre szabalyok utan: 
 predicate:  25 
 tripletek:  831288 
 átlag:  33251.52 
 szórás:  54688.9512974


In [1028]:
utils = Utils()
g_dot = utils.to_dot(graphs_zero[144])

In [1029]:
v_g = Source(g_dot)
print(g_dot)

digraph finite_state_machine {
	dpi=100;
	Alki_Larnaca_F_C__ [shape = circle, label = "Alki_Larnaca_F_C_"];
	help_140642071116560 [shape = circle, label = "help"];
	promote_140642071114000 [shape = circle, label = "promote"];
	something_140642071114576 [shape = circle, label = "something"];
	Alki_Larnaca_F_C__ -> help_140642071116560 [ label = "0" ];
	promote_140642071114000 -> help_140642071116560 [ label = "0" ];
	promote_140642071114000 -> something_140642071114576 [ label = "2" ];
}


In [ ]:
v_g

In [1031]:
triplets[144]

('2009\\u201310_Cypriot_Second_Division', 'promoted', 'Alki_Larnaca_F.C.')

In [ ]:
success_two

In [948]:
success_zero_manual = [1, 2, 3, 4, 7, 11, 15, 18, 20, 26, 27, 33, 36, 38 ,41, 42, 43, 48, 50, 52, 60, 61, 62, 
                       63, 65, 74, 75, 84, 85, 87, 89, 91, 104, 105, 116, 119, 125, 127, 129, 134, 138, 143, 147, 153, 155,
                      156, 161, 162, 167, 168, 179, 186, 189, 192, 194]

In [949]:
len(success_zero_manual)

55

In [950]:
print_statistics("Kezi kiertekeles 0 tipusra", [triplets[k][1] for k in success_zero_manual])

Kezi kiertekeles 0 tipusra 
 predicate:  55 
 tripletek:  7624815 
 átlag:  138633.0 
 szórás:  532524.389791


In [ ]:
success_two_manual = [2, 26, 33, 50, 60, 87, 104, 114, 116, 141, 158, 161, 167, 171, 179, 186, 189]

In [1034]:
len(success_two_manual)

16

In [989]:
print_statistics("Kezi kiertekeles 2 tipusra", [triplets[k][1] for k in success_two_manual])

Kezi kiertekeles 2 tipusra 
 predicate:  16 
 tripletek:  742349 
 átlag:  46396.8125 
 szórás:  62058.5405789


In [1033]:
len(neutral_zero)

24

In [1032]:
success_neutral_zero = [14,16,39,59,67,101,106,111,132,159,178,180]
len(success_neutral_zero)

12